In [1]:
pip install face_recognition opencv-python dlib numpy pillow matplotlib jupyter

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Gerekli kütüphaneleri import et
import face_recognition
import cv2
import numpy as np
from datetime import datetime
import os
import logging
import dlib
from PIL import Image
import matplotlib.pyplot as plt
import requests
import bz2
import shutil

%matplotlib inline

# Logging ayarları
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
def download_model(model_path: str) -> bool:
    """Model dosyasını indirir ve belirtilen konuma kaydeder"""
    try:
        url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
        temp_path = f"{model_path}.bz2"
        
        print(f"Model indiriliyor: {url}")
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        with open(temp_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        
        print("Model dosyası çıkartılıyor...")
        with bz2.open(temp_path, 'rb') as source, open(model_path, 'wb') as dest:
            shutil.copyfileobj(source, dest)
        
        os.remove(temp_path)
        print(f"Model başarıyla indirildi: {model_path}")
        return True
        
    except Exception as e:
        print(f"Model indirme hatası: {str(e)}")
        return False

def check_and_download_model():
    try:
        models_dir = os.path.join(os.getcwd(), 'models')
        os.makedirs(models_dir, exist_ok=True)
        
        model_path = os.path.join(models_dir, 'shape_predictor_68_face_landmarks.dat')
        if not os.path.exists(model_path):
            print("Model dosyası bulunamadı, indiriliyor...")
            if not download_model(model_path):
                raise RuntimeError("Model indirilemedi!")
        else:
            print("Model dosyası mevcut.")
            
        return model_path
    except Exception as e:
        print(f"Hata: {str(e)}")
        return None

# Model kontrolü yap
model_path = check_and_download_model()

Model dosyası mevcut.


In [4]:
class FaceRecognition:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []
        
    def load_known_face(self, image_path, person_name):
        """Bilinen bir yüzü yükle"""
        try:
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)
            
            if face_encodings:
                self.known_face_encodings.append(face_encodings[0])
                self.known_face_names.append(person_name)
                print(f"{person_name} başarıyla eklendi.")
                return True
            else:
                print(f"Uyarı: {image_path} dosyasında yüz bulunamadı!")
                return False
                
        except Exception as e:
            print(f"Hata: {str(e)}")
            return False
    
    def recognize_faces(self, image_path):
        """Görüntüdeki yüzleri tanı"""
        try:
            image = face_recognition.load_image_file(image_path)
            face_locations = face_recognition.face_locations(image)
            face_encodings = face_recognition.face_encodings(image, face_locations)
            
            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(
                    self.known_face_encodings,
                    face_encoding,
                    tolerance=0.6
                )
                name = "Bilinmeyen"
                
                if True in matches:
                    first_match_index = matches.index(True)
                    name = self.known_face_names[first_match_index]
                    
                face_names.append(name)
            
            # Görüntüyü göster
            self.display_results(image, face_locations, face_names)
            return face_names
            
        except Exception as e:
            print(f"Hata: {str(e)}")
            return []
    
    def display_results(self, image, face_locations, face_names):
        """Sonuçları görselleştir"""
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Yüz çerçevesi
            cv2.rectangle(image_rgb, (left, top), (right, bottom), (0, 255, 0), 2)
            
            # İsim etiketi
            cv2.putText(
                image_rgb,
                name,
                (left, top - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.75,
                (0, 255, 0),
                2
            )
        
        plt.figure(figsize=(12, 8))
        plt.imshow(image_rgb)
        plt.axis('off')
        plt.show()

# Yüz tanıma nesnesini oluştur
face_recognizer = FaceRecognition()

In [5]:
known_faces_dir = 'known_faces'
os.makedirs(known_faces_dir, exist_ok=True)

if os.path.exists(known_faces_dir):
    for filename in os.listdir(known_faces_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            name = os.path.splitext(filename)[0]
            image_path = os.path.join(known_faces_dir, filename)
            face_recognizer.load_known_face(image_path, name)

yusuf başarıyla eklendi.
ömer başarıyla eklendi.


In [6]:
def add_person(image_path, name):
    if face_recognizer.load_known_face(image_path, name):
        # Fotoğrafı known_faces klasörüne kopyala
        extension = os.path.splitext(image_path)[1]
        new_path = os.path.join(known_faces_dir, f"{name}{extension}")
        shutil.copy2(image_path, new_path)
        print(f"{name} başarıyla eklendi ve fotoğraf kaydedildi.")
    else:
        print("Kişi eklenemedi.")

# Örnek kullanım:
# add_person('path/to/photo.jpg', 'Ahmet')

In [7]:
def test_recognition(image_path):
    if os.path.exists(image_path):
        detected_faces = face_recognizer.recognize_faces(image_path)
        print(f"\nTanınan yüzler: {detected_faces}")
    else:
        print(f"Hata: {image_path} bulunamadı!")

# Örnek kullanım:
# test_recognition('test.jpg')

In [8]:
def test_webcam():
    cap = cv2.VideoCapture(0)
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            # BGR'den RGB'ye dönüştür
            rgb_frame = frame[:, :, ::-1]
            
            # Yüzleri bul ve tanı
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
            
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                matches = face_recognition.compare_faces(
                    face_recognizer.known_face_encodings,
                    face_encoding
                )
                name = "Bilinmeyen"
                
                if True in matches:
                    first_match_index = matches.index(True)
                    name = face_recognizer.known_face_names[first_match_index]
                
                # Yüzü çerçevele
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(
                    frame,
                    name,
                    (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.75,
                    (0, 255, 0),
                    2
                )
            
            # Görüntüyü göster
            cv2.imshow('Webcam', frame)
            
            # 'q' tuşuna basılırsa çık
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    finally:
        cap.release()
        cv2.destroyAllWindows()

# Webcam testi için bu satırı çalıştırın:
# test_webcam()

In [ ]:
import face_recognition
import cv2
import numpy as np
from datetime import datetime
import os
import logging
import dlib
from PIL import Image
import matplotlib.pyplot as plt
import requests
import bz2
import shutil
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk

class FaceRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Yüz Tanıma Sistemi")
        self.root.geometry("800x600")
        
        # Logging ayarları
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
        # Model kontrolü
        self.check_and_download_model()
        
        # Yüz tanıma sınıfını başlat
        self.face_recognizer = FaceRecognition()
        
        # GUI bileşenlerini oluştur
        self.create_gui()
        
        # Known faces klasörünü oluştur
        self.known_faces_dir = 'known_faces'
        os.makedirs(self.known_faces_dir, exist_ok=True)
        
        # Mevcut yüzleri yükle
        self.load_known_faces()

    def create_gui(self):
        # Ana frame
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # Kamera görüntüsü için canvas
        self.camera_canvas = tk.Canvas(main_frame, width=640, height=480)
        self.camera_canvas.grid(row=0, column=0, columnspan=2, pady=10)
        
        # Butonlar için frame
        button_frame = ttk.Frame(main_frame)
        button_frame.grid(row=1, column=0, columnspan=2, pady=10)
        
        # Butonlar
        ttk.Button(button_frame, text="Kişi Ekle", command=self.add_person_dialog).pack(side=tk.LEFT, padx=5)
        self.camera_button = ttk.Button(button_frame, text="Kamerayı Başlat", command=self.toggle_camera)
        self.camera_button.pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Fotoğraftan Tanı", command=self.recognize_from_photo).pack(side=tk.LEFT, padx=5)
        
        # Durum çubuğu
        self.status_var = tk.StringVar()
        self.status_var.set("Hazır")
        status_label = ttk.Label(main_frame, textvariable=self.status_var)
        status_label.grid(row=2, column=0, columnspan=2, pady=5)
        
        # Kamera değişkenleri
        self.camera_active = False
        self.cap = None

    def add_person_dialog(self):
        dialog = tk.Toplevel(self.root)
        dialog.title("Kişi Ekle")
        dialog.geometry("300x150")
        
        ttk.Label(dialog, text="İsim:").pack(pady=5)
        name_entry = ttk.Entry(dialog)
        name_entry.pack(pady=5)
        
        def select_photo():
            file_path = filedialog.askopenfilename(
                filetypes=[("Image files", "*.jpg *.jpeg *.png")]
            )
            if file_path:
                try:
                    self.add_person(file_path, name_entry.get())
                    dialog.destroy()
                except Exception as e:
                    messagebox.showerror("Hata", str(e))
        
        ttk.Button(dialog, text="Fotoğraf Seç", command=select_photo).pack(pady=10)

    def add_person(self, image_path, name):
        if not name:
            raise ValueError("Lütfen bir isim girin!")
            
        if self.face_recognizer.load_known_face(image_path, name):
            # Fotoğrafı known_faces klasörüne kopyala
            extension = os.path.splitext(image_path)[1]
            new_path = os.path.join(self.known_faces_dir, f"{name}{extension}")
            shutil.copy2(image_path, new_path)
            self.status_var.set(f"{name} başarıyla eklendi")
            messagebox.showinfo("Başarılı", f"{name} başarıyla eklendi")
        else:
            raise ValueError("Fotoğrafta yüz bulunamadı!")

    def toggle_camera(self):
        if not self.camera_active:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Hata", "Kamera açılamadı!")
                return
                
            self.camera_active = True
            self.camera_button.configure(text="Kamerayı Durdur")
            self.update_camera()
        else:
            self.camera_active = False
            self.camera_button.configure(text="Kamerayı Başlat")
            if self.cap:
                self.cap.release()

    def update_camera(self):
        if self.camera_active:
            ret, frame = self.cap.read()
            if ret:
                # Yüz tanıma işlemi
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
                
                for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                    matches = face_recognition.compare_faces(
                        self.face_recognizer.known_face_encodings,
                        face_encoding
                    )
                    name = "Bilinmeyen"
                    
                    if True in matches:
                        first_match_index = matches.index(True)
                        name = self.face_recognizer.known_face_names[first_match_index]
                    
                    # Yüzü çerçevele
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    cv2.putText(
                        frame,
                        name,
                        (left, top - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.75,
                        (0, 255, 0),
                        2
                    )
                
                # Görüntüyü canvas'a aktar
                self.photo = ImageTk.PhotoImage(image=Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
                self.camera_canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
            
            self.root.after(10, self.update_camera)

    def recognize_from_photo(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )
        if file_path:
            detected_faces = self.face_recognizer.recognize_faces(file_path)
            if detected_faces:
                messagebox.showinfo("Sonuç", f"Tanınan yüzler: {', '.join(detected_faces)}")
            else:
                messagebox.showinfo("Sonuç", "Yüz bulunamadı veya tanınamadı")

    def load_known_faces(self):
        if os.path.exists(self.known_faces_dir):
            for filename in os.listdir(self.known_faces_dir):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    name = os.path.splitext(filename)[0]
                    image_path = os.path.join(self.known_faces_dir, filename)
                    self.face_recognizer.load_known_face(image_path, name)

    def check_and_download_model(self):
        try:
            models_dir = os.path.join(os.getcwd(), 'models')
            os.makedirs(models_dir, exist_ok=True)
            
            model_path = os.path.join(models_dir, 'shape_predictor_68_face_landmarks.dat')
            if not os.path.exists(model_path):
                self.status_var.set("Model indiriliyor...")
                if not self.download_model(model_path):
                    raise RuntimeError("Model indirilemedi!")
            
            self.status_var.set("Model hazır")
            return model_path
        except Exception as e:
            self.logger.error(f"Hata: {str(e)}")
            messagebox.showerror("Hata", str(e))
            return None

    def download_model(self, model_path):
        try:
            url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
            temp_path = f"{model_path}.bz2"
            
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            with open(temp_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            with bz2.open(temp_path, 'rb') as source, open(model_path, 'wb') as dest:
                shutil.copyfileobj(source, dest)
            
            os.remove(temp_path)
            return True
            
        except Exception as e:
            self.logger.error(f"Model indirme hatası: {str(e)}")
            return False

class FaceRecognition:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []
        
    def load_known_face(self, image_path, person_name):
        try:
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)
            
            if face_encodings:
                self.known_face_encodings.append(face_encodings[0])
                self.known_face_names.append(person_name)
                return True
            return False
                
        except Exception as e:
            logging.error(f"Hata: {str(e)}")
            return False
    
    def recognize_faces(self, image_path):
        try:
            image = face_recognition.load_image_file(image_path)
            face_locations = face_recognition.face_locations(image)
            face_encodings = face_recognition.face_encodings(image, face_locations)
            
            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(
                    self.known_face_encodings,
                    face_encoding,
                    tolerance=0.6
                )
                name = "Bilinmeyen"
                
                if True in matches:
                    first_match_index = matches.index(True)
                    name = self.known_face_names[first_match_index]
                    
                face_names.append(name)
            
            return face_names
            
        except Exception as e:
            logging.error(f"Hata: {str(e)}")
            return []

def main():
    root = tk.Tk()
    app = FaceRecognitionApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

ERROR:__main__:Hata: 'FaceRecognitionApp' object has no attribute 'status_var'
